# Scraping data for Flight price predicition project:

In [1]:
# importing all required libraries for webscraping
import pandas as pd
import numpy as np
import selenium
import time
# importing selenium webdrivers
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from dateutil.relativedelta import relativedelta
# importing exceptions which need to be handled
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import requests
from bs4 import BeautifulSoup
import csv
import linecache
import sys
import warnings
warnings.filterwarnings('ignore')

* You have to scrape at least 1500 rows of data. You can scrape more data as well, it’s up to you, 
More the data better the model
In this section you have to scrape the data of flights from different websites (yatra.com, 
skyscanner.com, official websites of airlines, etc). The number of columns for data doesn’t have 
limit, it’s up to you and your creativity. Generally, these columns are                                                                  
1) airline name,                                                     
2) date of journey,                                                  
3) source,                                                           
4) destination,                                                       
5) meal,                                                            
6) departure time,                                                    
7) arrival time,                                                      
8) duration,                                                           
9) total stops and                                                    
10) the target variable price.                                         
You can make changes to it, you can add or you can remove some columns, it completely
depends on the website from which you are fetching the data.

# Extracting data of flights for 1500 or more rows

In [2]:
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
def scrap(origin,dest,t_date):
    global none
    Url = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ dest +"-"+ t_date +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E"

    try:
        driver = selenium.webdriver.Chrome(r"C:\\Users\\amitt\\Downloads\\chromedriver_win32\\chromedriver.exe") 
        print ("Required URL: " + Url)

        driver.get(Url)  
        print ("page found ...")

        element_xpath = '//*[@id="left-side--wrapper"]/div[2]'

        
        element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

        print ("Scroll to bottom ...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...")
        driver.quit()

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body)
        
        spanFlightNo = soupBody.find_all("span", 'boldFont blackText airlineName')
        print("spanFlightNo  = " + str(len(spanFlightNo)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.find_all("p", {"class": "darkText"})
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.find_all("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.find_all("p", {"class": "flightsLayoverInfo"})
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightNo,t_date,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(t_date," scraping complete")
        
        none =True 

In [4]:
#Lists to store scraped data
airlines=[]
travel_date=[]
source=[]
destination=[]
price=[]
dep_time=[]
arr_time=[]
stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airlines', 'Travel_date','From','To','Departure time','Arrival time','Stops','Price'])
flight_data.to_csv('Flightsdata.csv')
 
def transform(spanFlightNo,t_date,City,Time,spanFlightCost,pFlightStops):
    
    airlines.clear()
    travel_date.clear()
    price.clear()
    stops.clear()
    source.clear()
    dep_time.clear()
    destination.clear()
    arr_time.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightNo)):
        
        airlines.append(spanFlightNo[i].string)
        travel_date.append(t_date)
        price.append(spanFlightCost[i].string)
        stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            source.append(City[j].string)
            dep_time.append(Time[j].string)
        else:
            
            destination.append(City[j].string)
            arr_time.append(Time[j].string) 

            
            
    temp = {'Airlines':airlines,
            'Travel_date':travel_date,
            'From':source,
            'To':destination,
            'Departure time':dep_time,
            'Arrival time':arr_time,
            'Stops':stops,
            'Price':price}
    df=pd.DataFrame(temp)
    df.to_csv('Flightsdata.csv', mode='a', header=False)
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [5]:
dates=(pd.date_range('03/01/2022','05/31/2022',7))
date_1=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)
    month=str(date.month).zfill(2)
    year=str(date.year)
    date=day+'/'+month+'/'+year
    date_1.append(date)
print(date_1)

['01/03/2022', '16/03/2022', '31/03/2022', '15/04/2022', '30/04/2022', '15/05/2022', '31/05/2022']


In [6]:
list_of_dest=['DEL','BOM','PNQ','BLR','GOI','AMD']
for i in range(0,len(list_of_dest)):
    orgin=list_of_dest[i]
    for_to=list_of_dest
    print("--------------")
    for j in range(0,len(for_to)):
        dest=for_to[j]
        none = False
        print("origin = "+orgin)
        print("dest = "+dest+'\n')
        if orgin!=dest:
            for i,date in enumerate(date_1) :
                if none !=True:
                    scrap(orgin,dest,date)
                    print(" ")
                else:
                    break
            else: 
                print( "-----Skiped-----")
                print(" ")

--------------
origin = DEL
dest = DEL

origin = DEL
dest = BOM

Required URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-01/03/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
page found ...
Scroll to bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightNo  = 89
Time Count in Scraping  = 178
City Count in Scraping  = 179
spanFlightCost Count in Scraping  = 89
pFlightStops Count in Scraping  = 89
Transaform Function Running.... 


,Airlines,Travel_date,From,To,Departure time,Arrival time,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
2,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
3,AirAsia,01/03/2022,New Delhi,Mumbai,20:00,02:25,1 stop via Goa,"₹ 5,953"
4,AirAsia,01/03/2022,New Delhi,Mumbai,12:20,02:25,1 stop via Goa,"₹ 5,953"
...,...,...,...,...,...,...,...,...
85,SpiceJet,01/03/2022,New Delhi,Mumbai,07:45,12:25,1 stop via Jabalpur,"₹ 9,630"
86,Vistara,01/03/2022,New Delhi,Mumbai,13:20,18:20,1 stop via Varanasi,"₹ 10,785"
87,IndiGo,01/03/2022,New Delhi,Mumbai,18:00,22:25,1 stop via Indore,"₹ 10,838"
88,IndiGo,01/03/2022,New Delhi,Mumbai,05:15,10:05,1 stop via Ahmedabad,"₹ 11,153"


Lenght :  90
 
Required URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-16/03/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
page found ...
Scroll to bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightNo  = 89
Time Count in Scraping  = 178
City Count in Scraping  = 180
spanFlightCost Count in Scraping  = 89
pFlightStops Count in Scraping  = 89
Transaform Function Running.... 


,Airlines,Travel_date,From,To,Departure time,Arrival time,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
2,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
3,AirAsia,01/03/2022,New Delhi,Mumbai,20:00,02:25,1 stop via Goa,"₹ 5,953"
4,AirAsia,01/03/2022,New Delhi,Mumbai,12:20,02:25,1 stop via Goa,"₹ 5,953"
...,...,...,...,...,...,...,...,...
174,Vistara,16/03/2022,New Delhi,Mumbai,05:55,11:25,1 stop via Bengaluru,"₹ 6,095"
175,Vistara,16/03/2022,New Delhi,Mumbai,16:05,21:50,1 stop via Bengaluru,"₹ 6,095"
176,Vistara,16/03/2022,New Delhi,Mumbai,17:35,23:25,1 stop via Bengaluru,"₹ 6,095"
177,Vistara,16/03/2022,New Delhi,Mumbai,17:40,23:00,1 stop via Hyderabad,"₹ 6,263"


Lenght :  179
 
Required URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-31/03/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
page found ...
Scroll to bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightNo  = 87
Time Count in Scraping  = 174
City Count in Scraping  = 176
spanFlightCost Count in Scraping  = 87
pFlightStops Count in Scraping  = 87
Transaform Function Running.... 


...

Lenght :  7942
 
Required URL: https://www.makemytrip.com/flight/search?itinerary=AMD-GOI-15/05/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
page found ...
Scroll to bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightNo  = 15
Time Count in Scraping  = 30
City Count in Scraping  = 31
spanFlightCost Count in Scraping  = 15
pFlightStops Count in Scraping  = 15
Transaform Function Running.... 


,Airlines,Travel_date,From,To,Departure time,Arrival time,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
2,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
3,AirAsia,01/03/2022,New Delhi,Mumbai,20:00,02:25,1 stop via Goa,"₹ 5,953"
4,AirAsia,01/03/2022,New Delhi,Mumbai,12:20,02:25,1 stop via Goa,"₹ 5,953"
...,...,...,...,...,...,...,...,...
7952,Go First,15/05/2022,Ahmedabad,Goa,09:25,16:45,1 stop via Mumbai,"₹ 5,210"
7953,IndiGo,15/05/2022,Ahmedabad,Goa,18:30,22:55,1 stop via Mumbai,"₹ 5,665"
7954,IndiGo,15/05/2022,Ahmedabad,Goa,20:50,01:30,1 stop via Pune,"₹ 5,773"
7955,IndiGo,15/05/2022,Ahmedabad,Goa,08:50,13:50,1 stop via Hyderabad,"₹ 6,224"


Lenght :  7957
 
Required URL: https://www.makemytrip.com/flight/search?itinerary=AMD-GOI-31/05/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
page found ...
Scroll to bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightNo  = 39
Time Count in Scraping  = 78
City Count in Scraping  = 78
spanFlightCost Count in Scraping  = 39
pFlightStops Count in Scraping  = 39
Transaform Function Running.... 


,Airlines,Travel_date,From,To,Departure time,Arrival time,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
2,AirAsia,01/03/2022,New Delhi,Mumbai,08:20,14:10,1 stop via Ranchi,"₹ 5,953"
3,AirAsia,01/03/2022,New Delhi,Mumbai,20:00,02:25,1 stop via Goa,"₹ 5,953"
4,AirAsia,01/03/2022,New Delhi,Mumbai,12:20,02:25,1 stop via Goa,"₹ 5,953"
...,...,...,...,...,...,...,...,...
7991,IndiGo,31/05/2022,Ahmedabad,Goa,08:50,18:15,1 stop via Mumbai,"₹ 6,003"
7992,Go First,31/05/2022,Ahmedabad,Goa,09:10,16:55,1 stop via Bengaluru,"₹ 7,290"
7993,Vistara,31/05/2022,Ahmedabad,Goa,08:25,13:50,1 stop via New Delhi,"₹ 7,313"
7994,Go First,31/05/2022,Ahmedabad,Goa,22:20,13:20,1 stop via New Delhi,"₹ 7,493"


Lenght :  7996
 
-----Skiped-----
 
origin = AMD
dest = AMD

